In [4]:
!pip install opencv-python-headless==4.1.2.30

     |████████████████████████████████| 21.8 MB 1.9 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.64
    Uninstalling opencv-python-headless-4.5.5.64:
      Successfully uninstalled opencv-python-headless-4.5.5.64


In [51]:
from craft_text_detector import (
    read_image,
    load_craftnet_model,
    load_refinenet_model,
    get_prediction,
    export_detected_regions,
    export_extra_results,
    empty_cuda_cache
)
import easyocr
from PIL import Image
import numpy as np
import re
import io
from scipy.ndimage import interpolation as inter
import cv2

reader = easyocr.Reader(['en'], gpu=True)
refine_net = load_refinenet_model(cuda=True)
craft_net = load_craftnet_model(cuda=True)

In [66]:
image = np.array(Image.open('/content/СТС_18.jpeg'))

h,w = image.shape[:2]
print(h,w)
img = cv2.resize(image, (2700, 3000), fx=0.5, fy=0.3)
image = read_image(img)


prediction_result = get_prediction(
    image=image,
    craft_net=craft_net,
    refine_net=refine_net,
    text_threshold=0.7,
    link_threshold=0.4,
    low_text=0.4,
    cuda=True,
    long_size=1280
)

idx = 0
ocr_text = []
boxes = prediction_result["boxes"]
for box in boxes[4:]:
    idx+=1
    x,y,w,h = cv2.boundingRect(box)
    resize_image = image[y:y+h, x:x+w]
    result = reader.readtext(resize_image, detail=0)
    ocr_text.append(result)
print(ocr_text)


923 687


/usr/local/lib/python3.7/dist-packages/craft_text_detector/craft_utils.py:415: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  polys = np.array(polys)
/usr/local/lib/python3.7/dist-packages/craft_text_detector/predict.py:110: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  polys_as_ratio = np.array(polys_as_ratio)


[['MaenruduraunonHslf', 'uomep', '(VIN )'], ['WAUZZZ8V3J1075219'], ['Mapra,', 'MOACAL', 'AyAM A?'], ['AUd A3'], ['Tun', 'TTC'], ['TiL', 'ELU', 'Kareropu lef&ePk EcBa'], ['"pHuen'], ['B'], ['2018'], ['FoA', 'BEIyCKa', 'TC'], ['Maccu', '(paMa)', 'No', 'OTCYTCTBYET'], ['Ky3oB', '(Kabuua', 'upuuen', 'No'], ['WAUZZZ8V3J1075219'], ['Uee', 'BEnBIV'], ['MouuocTb', 'ABura TexA.', 'KBT/^.', '0.110.3/150'], ['3KOAOTHHCCKHM', 'Kiacc NATbIM'], ['Hacnopr', 'TC', 'cepua 78 YX No 478063'], ['Paxpemenua a', 'max', 'Macca,', '1795', 'Kq'], ['1385'], ['Macca', '6e3', '#arpy3Ku,', 'kq'], ['77', '61', 'N', '164232']]


In [44]:
image = np.array(Image.open('/content/ПТС_13.jpeg'))
result = reader.readtext(image, detail=0)
print(result)

['IACIIOPT TPAHCIIOPTHOTO CPEACTBA', '78', 'JX', '294058', 'MACHTUbuKIQUIOHIILII', "#omep' (VIN)", 'WAUZLZBUIHR483449', 'Mupka,', "Moe 'C", 'AUDI QX', 'Iluuhciioimuiiiio', '(un \'"TC) JIFTKOBOn', 'Jcodble', 'ot.etku', "Kareropux 'IC", '(4', '#pIIICH )', '5. To,', "MBTOTOACHuIA 'IY", '201', 'MoAelb', 'NNu', 'AATGAS', 'CZD 296.425', '7. Maccu', '([YI,MIU )', 'Nv', '0TC YTC TBYET', '8. Kyaor   (Kadua', 'MpHUCHT )', 'No', 'WAUZZZSUIHRU83449', 'VTHJIZILMIOHHEIT coop', '9. Muerr', 'KJOHa', '(Kagubl , HQHIICTa )', 'YRHBI', 'ynayen: TIIO 1', 'MouHocb  BHTUTCAA', '(KBt)', '150', '7110)', '10009194/040517/', '1395', '4. Pado"uit', 'O6LOM', 'ABMATCAX,', 'KV6.', 'CM', 'TC-61005S72', '12.', "'Tun", 'AurarexA', 'BIHSHBOBBQI', '13.', 'HK01OTHUICCKHii', 'AAucc', 'DATLIA', 'YBDOC Ne', '8970177000006276516', '14. Paapeenuax', 'MaKCHALAAbHta%', 'MuCcu', 'KT', '2003', '1537', '15.', 'Macca 6e3 Harpy3Ku;', "16 . Waroronureab 'TC", 'C"paHu )', 'AUDIAG', '@CHAHES)', '17. OAoopenue', 'TII', '"TY', 'No IC RU',